In [0]:
%pip install -r requirements.txt
%restart_python

In [0]:
%restart_python

In [0]:
from src.ingest_data import load_and_prepare_data
from src.evaluate_model import evaluate_models_on_test
import json
import pandas as pd
from pathlib import Path
import mlflow
import random
import numpy as np


# Set seeds
random.seed(42)
np.random.seed(42)

In [0]:
CONFIG_PATH = Path("configs/")

def run_all_models_evaluation():
    all_forecasts = []
    for config_file in CONFIG_PATH.glob("*.json"):

        # Skip evaluation of horizon 1 since it is technically 2-month horizon forecast
        if config_file.name.endswith('_h1.json'):
            print(f"Skipping config {config_file.name} (horizon 1)")
            continue

        with open(config_file) as f:
            params = json.load(f)

        model_name = params["model_name"]
        target = params['target']
        horizon = params['horizon']

        print(f"Running pipeline for: {model_name}")

        # Load and prepare data
        df = load_and_prepare_data(
            path="data/processed/combined_total_household_data_interpolate.csv",
            target=target,
            params=params
        )

        # Define output path per model + horizon for plot & CSV saving
        output_path = Path(f"outputs/validations/{model_name}")

        # Train the model and evaluate
        rmse, mape, max_error, results = evaluate_models_on_test(df, h=horizon, param_dict=params, target = target, output_path = output_path)

        # Collect summary info for this run
        all_forecasts.append({
            "model_name": model_name,
            "target": target,
            "horizon": horizon,
            "rmse": rmse,
            "mape": mape,
            "max_error": max_error
        })

    # Combine all forecast summaries and save
    summary_df = pd.DataFrame(all_forecasts)
    summary_path = Path("outputs/validations/all_forecasts_summary.csv")
    summary_path.parent.mkdir(parents=True, exist_ok=True)
    summary_df.to_csv(summary_path, index=False)
    print(f"Saved all forecasts summary to: {summary_path}")

    return summary_df


In [0]:
summary_df = run_all_models_evaluation()
summary_df